# Introduction


RoGemma-7B-Instruct is a Large Language Model that was developed in the frame of the project [**LLM for Romanian**](https://ilds.ro/llm-for-romanian/). It is derived from Gemma-7B-Instruct and is intended to be used for research and applications using Romanian language. It can be accesed on HuggingFace: [OpenLLM-Ro/RoGemma-7b-Instruct](https://huggingface.co/OpenLLM-Ro/RoGemma-7b-Instruct).

## What is Gemma?

Gemma is a collection of lightweight, text-to-text, decoder-only large language models. It has many model variations, including versions for Keras, PyTorch,Transformers, Gemma C++, GGUF, LiteRT, TensoRT-LLM, MaxText and more frameworks, and sizes (2B, and 7B).

## What is LLM for Romanian?

LLM for Romanian is a project developed by the Institute for Logic and Data Science, for pre-training and fine-tuning of Large Language Models to obtain a foundation model for the Romanian language.




# Preprequisites


Let's install the libraries and import the packages we will need to test this model.

In [1]:
!pip install -U transformers accelerate einops xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [2]:
from time import time
from IPython.display import display, Markdown
from torch import cuda, bfloat16
from transformers import BitsAndBytesConfig, AutoConfig, AutoTokenizer, AutoModelForCausalLM 

# Initialize the model

Because we are not able to initialize and run the model as it is with Kaggle GPUs, we will initialize directly the quantized model.   
We specify the path to the HuggingFace model.  
We set the 4bit quantization configuration using bitsandbytes.

In [3]:
model_id = 'OpenLLM-Ro/RoGemma-7b-Instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

We set the model configuration.  
Then we load the HuggingFace model with the option to quantize it (`quantization_config`=`bnb_config`).  
First the model shards are downloaded from HuggingFace.  

In [4]:
time_1 = time()
model_config = AutoConfig.from_pretrained(
    model_id,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
time_2 = time()
print(f"Download and quantize model: {round(time_2-time_1, 3)} sec.")

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Download and quantize model: 590.732 sec.


In [5]:
time_1 = time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare tokenizer: {round(time_2-time_1, 3)} sec.")

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Prepare tokenizer: 2.484 sec.


# Test the model

Let's quickly test the model with a simple query.

In [6]:
system_message = "Ești un instructor inteligent care poate să răspundă la întrebări de cultură generală in limba română"

instruction = "Care e cel mai înalt munte din România?"

chat = [
        {"role": "user", "content": instruction},
        ]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, system_message=system_message)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<bos><start_of_turn>user
Care e cel mai înalt munte din România?<end_of_turn>
<start_of_turn>model
Cel mai înalt munte din România este Muntele Moldoveanu, care se află la o altitudine de 2.544 metri (8.346 ft). Este situat în Munții Retezat din Parcul Național Retezat din Munții Parâng din Carpații Meridionali.<eos>


Let's refine a bit the query, let's write a query function.

In [7]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        #{"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=False,
        system_message=system_message
        )
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    inputs = tokenizer.encode(prompt,
                             add_special_tokens=False,
                             return_tensors="pt")
    
    outputs = model.generate(input_ids=inputs,
                             temperature=temperature,
                             max_new_tokens=max_length)
    
    answer = tokenizer.decode(outputs[0])
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
Ești un instructor inteligent care poate să răspundă la întrebări de cultură generală in limba română
"""

# Test with the query function

Let's add also a function to beautify the answer.

In [8]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [9]:
user_message = "Care e capitala Romaniei?"
response = query_model(system_message, user_message, 0.1, 32)
display(Markdown(colorize_text(response)))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




**<font color='red'>Question:</font>** Care e capitala Romaniei? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Care e capitala Romaniei? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Capitala României este București.<eos> 

**<font color='magenta'>Total time:</font>** 1.11 sec.

In [10]:
user_message = "Cine este primarul Bucurestiului?"
response = query_model(system_message, user_message, 0.1, 32)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine este primarul Bucurestiului? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Cine este primarul Bucurestiului? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Primarul orașului București este Nicușor Dan.<eos> 

**<font color='magenta'>Total time:</font>** 1.69 sec.

In [11]:
user_message = "Cine a fost Ion Ghica?"
response = query_model(system_message, user_message, 0.1, 256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine a fost Ion Ghica? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Cine a fost Ion Ghica? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Ion Ghica a fost un om de stat și diplomat român care a servit ca prim-ministru al României de mai multe ori între 1852 și 1866. El a jucat un rol esențial în dezvoltarea și modernizarea țării în timpul domniei lui Alexandru al II-lea al Moldovei și a lui Gheorghe al V-lea al Țării Românești. Ghica a fost, de asemenea, implicată în înființarea primului guvern centralizat al României și în înființarea primului sistem bancar al țării.<eos> 

**<font color='magenta'>Total time:</font>** 11.28 sec.

In [12]:
user_message = "Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti"
response = query_model(system_message, user_message, 0.1, 128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
- Turnul din Catedrala Neapole
- Muzeul Național de Artă al României
- Piața din Piața Unirii<eos> 

**<font color='magenta'>Total time:</font>** 3.17 sec.

In [13]:
user_message = "Ce boarding games pot sa joc cu prietenii diseara?"
response = query_model(system_message, user_message, 0.1, 128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Ce boarding games pot sa joc cu prietenii diseara? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Ce boarding games pot sa joc cu prietenii diseara? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Există mai multe jocuri de masă pe care le puteți juca cu prietenii în seara asta. Unele opțiuni populare includ:

1. Monopoly: Un clasic de bord care implică cumpărarea și dezvoltarea de proprietăți, și încercarea de a ruina afacerile celorlalți jucători.
2. Cluedo: Un joc de detectiv în care jucătorii încearcă să rezolve misterul uciderii unui personaj prin excluderea diferitelor suspiciuni și arme.
3. Catan: Un joc de dezvoltare a resurselor în 

**<font color='magenta'>Total time:</font>** 10.9 sec.

# Save the model and tokenizer

We save the model and tokenizer, so that we can add the model as a Kaggle Model.


In [14]:
model.save_pretrained("/kaggle/working/RoGemma-7b-Instruct-v1-q4b")
tokenizer.save_pretrained("/kaggle/working/RoGemma-7b-Instruct-v1-q4b")

('/kaggle/working/RoGemma-7b-Instruct-v1-q4b/tokenizer_config.json',
 '/kaggle/working/RoGemma-7b-Instruct-v1-q4b/special_tokens_map.json',
 '/kaggle/working/RoGemma-7b-Instruct-v1-q4b/tokenizer.model',
 '/kaggle/working/RoGemma-7b-Instruct-v1-q4b/added_tokens.json',
 '/kaggle/working/RoGemma-7b-Instruct-v1-q4b/tokenizer.json')